In [10]:
from utils import load_results_policies
from games import get_game
import matplotlib.pyplot as plt
import ipywidgets as wid
import numpy as np
import seaborn as sns
from itertools import product
sns.set()


In [11]:
@wid.interact(p=(0, 1, 0.1), a=(1, 10, 1), step_type=['lola','naive'])
def plot_all_res(p=0.5, a=3, step_type='naive'):
    save_folder = "DistInfComms_p{}_a{}_{}_0".format(p, a, step_type)
    game = get_game("DistInfComms", p=p, a=a)
    _, _, p1s, p2s = load_results_policies(save_folder)
    p1s = np.array(p1s)
    p2s = np.array(p2s)
    
    ### Mutual information between signals and types, and signals and actions
    plt.figure(figsize=(12, 12))
    MIts = [game.MI_ts(p1, p2) for p1, p2 in zip(p1s, p2s)]
    MIas = [game.MI_as(p1, p2) for p1, p2 in zip(p1s, p2s)]

    ind = 0
    titles = ["I(t1;s1)", "I(t2;s2)", "I(a1;(s1, s2))", "I(a2;(s1, s2))"]
    for MIs in [MIts, MIas]:
        MI1s, MI2s = [ i for i, j in MIs ], [ j for i, j in MIs]
        for MI in [MI1s, MI2s]:
            ind += 1
            plt.subplot(2, 2, ind)
            plt.xlim(0,.7)
            plt.ylim(0, 40)
            plt.xlabel(titles[ind-1])
            sns.histplot(x=MI, binwidth=0.1)#, kde=True)
    plt.tight_layout()

interactive(children=(FloatSlider(value=0.5, description='p', max=1.0), IntSlider(value=3, description='a', ma…

In [12]:
@wid.interact(a=(1, 10, 1), step_type=['lola','naive'])
def plot_all_res(a=3, step_type='naive'):
    n_babble = []
    n_coord = []
    n_leader = []
    n_comm = []
    ps = [i/10 for i in range(0,11)]
    for p in ps:
        save_folder = "DistInfComms_p{}_a{}_{}_0".format(p, a, step_type)
        game = get_game("DistInfComms", p=p, a=a)
        _, _, p1s, p2s = load_results_policies(save_folder)
        p1s = np.array(p1s)
        p2s = np.array(p2s)
        
        ### Mutual information between signals and types, and signals and actions
        MIts = [game.MI_ts(p1, p2) for p1, p2 in zip(p1s, p2s)]
        MIas = [game.MI_as(p1, p2) for p1, p2 in zip(p1s, p2s)]

        ### Equilibrium with no information shared through messages (Babbling or Coordination)
        all_ps = set(range(40))
        no_info1 = set([i for i in range(len(MIts)) if MIts[i][0] < 0.1])
        no_info2 = set([i for i in range(len(MIts)) if MIts[i][1] < 0.1])
        info1 = set([i for i in range(len(MIts)) if MIts[i][0] > 0.1])
        info2 = set([i for i in range(len(MIts)) if MIts[i][1] > 0.1])
        ig_sig1  = set([i for i in range(len(MIas)) if MIas[i][0] < 0.1])
        ig_sig2  = set([i for i in range(len(MIas)) if MIas[i][1] < 0.1])
        sig1 = set([i for i in range(len(MIas)) if MIas[i][0] > 0.1])
        sig2 = set([i for i in range(len(MIas)) if MIas[i][1] > 0.1])

        no_info = no_info1.intersection(no_info2)
        info = info1.intersection(info2)
        ig_sig  = ig_sig1.intersection(ig_sig2)
        sig = sig1.intersection(sig2)

        babble = no_info.intersection(ig_sig)
        n_babble.append(len(babble))

        coord = no_info.intersection(sig)
        n_coord.append(len(coord))

        leader = no_info1.intersection(info2.intersection(sig1))
        n_leader.append(len(leader))

        comm = info
        n_comm.append(len(comm))
    
    all_ys = n_babble + n_coord + n_leader + n_comm
    all_xs = ps + ps + ps + ps
    names = ["Babble", "Coord", "Leader", "Comm"]
    hues = [names[int(i/len(ps))] for i in range(len(all_xs))]

    plt.figure(figsize=(12, 4))
    plt.ylim(0,25)
    plt.xlim(0,1)
    plt.ylabel("Count")
    plt.xlabel("p")
    sns.barplot(x=all_xs, y=all_ys, hue=hues)


interactive(children=(IntSlider(value=3, description='a', max=10, min=1), Dropdown(description='step_type', in…